In [1]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import subprocess
import mne
import mne_qt_browser
from scipy.stats import skew, kurtosis

# execute ls, sort files by date created
output = subprocess.run(["ls -t ../data_named"], stdout=subprocess.PIPE, shell=True, text=True)
# show only the last 5 files, line by line
files = output.stdout.split("\n")[:-1][:5]
filename = None
i = 0
print("\n>> Select file to analyze:\n")
for file in files:
    print("  ", i, ":", file)
    i += 1
index = input("\n>> ")
if index == "":
    index = 0 # if no input, default to the most recent file
    filename = files[index]
    print("\nOK, most recent file: ", filename)
else:
    index = int(index)
    filename = files[index]
    print("\nOK, opening: ", filename)


npz_file_path = f"../data_named/{filename}"

# Load the .npz file
data = np.load(npz_file_path, allow_pickle=True)

# Get the list of field names in the archive
field_names = data.files
print(f"Field names: {field_names}")
index = 6
window = []

# Access and print each field
for field in field_names:
    if int(field) == index:
        print(f"Field: {field}")
        field_data = data[field]
        label_value = field_data.item()['label']
        window = field_data.item()['data']
        print(f"Label: {label_value}")
        print(f"Window: {window}")
        continue


# window is a list of rows
df = pd.DataFrame(window)
# remove first two columns (millis and hall)
df = df.iloc[:, 2:]
# update header: ch0, ch1, ch2, ch3, ch4
df.columns = ["ch0", "ch1", "ch2", "ch3", "ch4"]
print(df.head())

data = df.values
data = data.T
data = data.astype(float)
#data = mne.filter.notch_filter(data, Fs=200, freqs=[30, 60, 90], verbose=True)

mne.set_config('MNE_BROWSER_BACKEND', 'qt')

ch_names = ['ch0', 'ch1', 'ch2', 'ch3', 'ch4']

sfreq = 1000  # Replace with your actual sampling frequency
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types='eeg')
raw = mne.io.RawArray(data, info)
raw.plot(show_scrollbars=True, block=False, scalings="auto")



>> Select file to analyze:

   0 : 0210_Jerry.npz

OK, opening:  0210_Jerry.npz
Field names: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62']
Field: 6
Label: tap
Window: [[4096, 0, 83, 561, 4095, 781, 4095], [4096, 0, 53, 496, 4095, 858, 4095], [4096, 0, 112, 286, 4095, 939, 4095], [4096, 0, 64, 160, 4095, 999, 4095], [4096, 0, 235, 144, 4095, 1002, 4095], [4096, 0, 95, 218, 4095, 994, 4095], [4096, 0, 54, 170, 4095, 948, 4095], [4096, 0, 131, 95, 4095, 996, 4095], [4096, 0, 85, 128, 4095, 1019, 4095], [4096, 0, 80, 288, 4095, 955, 4095], [4096, 0, 109, 408, 4095, 855, 4095], [4096, 0, 73, 501, 4095, 837, 4095], [4096, -8, 246, 448, 4095, 827, 4095], [4096, 0, 143, 331, 

Channels marked as bad:
none
